In [23]:
import fml_lib
import numpy as np
import pandas as pd
import sys
import math
from math import sqrt, exp, log
import matplotlib.pyplot as plt

Q1: A strategy exhibits a high turnover, high leverage, and high number of bets, with
a short holding period, low return on execution costs, and a high Sharpe ratio.
Is it likely to have large capacity? What kind of strategy do you think it is?

A: I think this is a high frequency strategy. I  think it's likely to have a high capacity as it has a low return on execution costs, implying higher transaction costs

Exercise 2: On the dollar bars dataset for E-mini S&P 500 futures

In [24]:
dollar_bars = pd.read_csv("USA500IDXUSD_bars.csv", parse_dates=[['Date', 'Timestamp']])
dollar_bars = dollar_bars.set_index("Date_Timestamp")
close_prices = dollar_bars['Close']
absolute_returns = close_prices.diff()
returns = absolute_returns/close_prices[:-1]
returns = returns.iloc[1:]
print(returns)

C:\Users\dstek\AppData\Local\Temp\ipykernel_40252\1214665056.py:1: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  dollar_bars = pd.read_csv("USA500IDXUSD_bars.csv", parse_dates=[['Date', 'Timestamp']])


Date_Timestamp
2023-07-02 22:00:10    0.000180
2023-07-02 22:00:20   -0.000071
2023-07-02 22:00:30   -0.000110
2023-07-02 22:00:40    0.000110
2023-07-02 22:00:50    0.000088
                         ...   
2024-01-25 03:57:00   -0.000106
2024-01-25 03:57:10    0.000000
2024-01-25 03:57:20    0.000003
2024-01-25 03:58:30   -0.000054
2024-01-25 03:58:40         NaN
Name: Close, Length: 840952, dtype: float64


2.a: HHI index on positive returns

In [25]:
pos_hhi = fml_lib.getHHI(returns[returns >=0])
print(pos_hhi)

2.9228947493621116e-06


2.b: HHI index on negative returns.

In [26]:
neg_hhi = fml_lib.getHHI(returns[returns < 0])
print(neg_hhi)

2.9738909372884605e-06


2.c: HHI index on time between bars

In [27]:
mid_bar_hhi = fml_lib.getHHI((returns.groupby(pd.Grouper(freq='M')).count()))
print(mid_bar_hhi)

0.004094296074964314


C:\Users\dstek\AppData\Local\Temp\ipykernel_40252\3726927066.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  mid_bar_hhi = fml_lib.getHHI((returns.groupby(pd.Grouper(freq='M')).count()))


2.d,2.e: The 95-percentile DD and TuW

In [28]:
dd, tuw = fml_lib.computeDD_TuW(returns)
percentile_dd = dd.quantile(.95, interpolation='nearest')
percentile_tuw = tuw.quantile(.95, interpolation='nearest')
print(f"95th percentile for drawdown is {percentile_dd}")
print(f"95th percentile for time under water is {percentile_tuw*365} days")

95th percentile for drawdown is 2.6625998516537033
95th percentile for time under water is 82.64583333333333 days


2.f: Annualized average return

2.g,2.h: Average return from hits and misses

In [29]:
avg_hit_return = np.mean(returns[returns>=0])
print(f"Average hit return is {avg_hit_return}")
avg_miss_return = np.mean(returns[returns<0])
print(f"Average miss return is {avg_miss_return}")

Average hit return is 6.172033830529491e-05
Average miss return is -6.618366463121042e-05


2.i: Annualized sharpe ratio

In [30]:
sr = np.mean(returns)/np.std(returns)
yearly_index = returns.index.year
yearly_returns = yearly_index.value_counts()
return_count = 0
year_count = 0
for year in yearly_returns:
    return_count = return_count + year
    year_count = year_count + 1
avg_yearly_return = return_count/year_count
annualized_sr = sr/avg_yearly_return
print(f"Annualized sharpe ratio is {annualized_sr}")

Annualized sharpe ratio is 2.559811465115836e-09


2.j: Information ratio, where the benchmark is the risk-free rate

In [31]:
# current 10 year yield is 4.2% so that's the risk free rate
